In [2]:
from langgraph.graph import StateGraph,START,END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [3]:
class state(TypedDict):
    input:str
    s_1=str
    s_2=str
    s_3=str

In [ ]:
def step_1(state:state):
    print("Step_1 executed")
    return {'s_1':'done','input':state['input']}

def step_2(state:state):
    print('Step_3 Hanging.....interrupting manually')
    time.sleep(30)
    return {'s_2':'done'}

def step_3(state:state):
    print('Step 3 executed')
    return {'s_3':'done'}

In [6]:
graph=StateGraph(state)

graph.add_node('step_1',step_1)
graph.add_node('step_2',step_2)
graph.add_node('step_3',step_3)

graph.add_edge(START,'step_1')
graph.add_edge('step_1','step_2')
graph.add_edge('step_2','step_3')
graph.add_edge('step_3',END)

check_pointer=InMemorySaver()
wf=graph.compile(checkpointer=check_pointer)

In [9]:
try:
  print("Running graph")
  wf.invoke({'input':'start'},config={"configurable":{"thread_id":"1"}})
except KeyboardInterrupt:
    print("Kernel manually interrupted")

Running graph
Step_1 executed
Hanging.....interrupting manually
Kernel manually interrupted


In [11]:
wf.get_state(config={"configurable":{"thread_id":"1"}})

StateSnapshot(values={'input': 'start'}, next=('step_2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06efc2-cbcc-6f33-8001-187b65cda45b'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}, 'thread_id': '1'}, created_at='2025-08-01T17:23:00.987064+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06efc2-cbc9-61a2-8000-54665a6f04a2'}}, tasks=(PregelTask(id='ab857c4b-0f48-1442-d35d-884058514493', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [12]:
final=wf.invoke(None,config={"configurable":{"thread_id":"1"}})

Hanging.....interrupting manually
Step 3 executed


In [14]:
print(final)

{'input': 'start'}
